In [ ]:
# SPDX-FileCopyrightText: © 2021 Hyunkyeong Ko <hyunkyeng925@naver.com>
 
# SPDX-License-Identifier: BSD-3-Clause

# 다음 뉴스 스크래핑
* 2021년 7월 ~ 2021년 9월

In [1]:
!pip install chromedriver-autoinstaller
!pip install selenium

In [13]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

import pandas as pd
import time
from datetime import datetime

date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

In [14]:
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0] 

try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')

driver.implicitly_wait(10)

In [ ]:
# 위에 코드 에러 나면 이 코드 실행해주세요.

#import shutil
#from selenium.webdriver.chrome.options import Options
#import subprocess

#try:
#    shutil.rmtree(r"c:\chrometemp") 
#except FileNotFoundError:
#    pass

#subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') 

#option = Options()
#option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

#chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
#try:
#    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
#except:
#    chromedriver_autoinstaller.install(True)
#    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
#driver.implicitly_wait(10)

In [15]:
def get_url_list():
    for i in range(20210701, 20210732):
        etl_ymd = str(i)
        url.append('https://search.daum.net/search?nil_suggest=btn&w=news&DA=PGD&q=%EC%BD%94%EB%A1%9C%EB%82%98&p=1&period=u&sd='+etl_ymd+'000000&ed='+etl_ymd+'235959')
    return url

url = []
get_url_list()

def get_article_url():
    
    urls_list = []
    
    for i in tqdm(range(1, 11)):
        try:
            order_num = str(i)
            article_url = driver.find_elements_by_xpath('//*[@id="newsColl"]/div[1]/ul/li['+order_num+']/div[2]/span[1]/a')
            urls_list.append(article_url[0].get_attribute('href'))    
            time.sleep(3)

        except:
            continue
        
    return urls_list

heads_list = []

def get_article_head():
    
    try:
        article_head = driver.find_elements_by_css_selector('#cSub > div > h3')[0].text
        time.sleep(1)
    except:
        article_head = ' '
        pass
    
    return article_head

dates_list = []

def get_article_date():
    
    try:
        html = driver.page_source
        dom = bs(html, 'lxml')
        article_date = dom.find("span", "num_date").text 
        time.sleep(1)
    except:
        article_date = ' '
        pass
    
    return article_date

press_list = []

def get_press_name():

    try:
        press_name = driver.find_elements_by_css_selector('#cSub > div > em > a > img')[0].get_attribute("alt")
        time.sleep(1)
    except:
        press_name = ' '
        pass
    
    return press_name

contents_list = []

def get_article_contents():
    
    try:
        article_content = driver.find_elements_by_xpath('//*[@id="harmonyContainer"]')[0].text
        time.sleep(1)
    except:
        article_content = ' '
        pass
        
    return article_content

centiments_like_list, centiments_angry_list = [], []

def get_article_centiments():
        
    try:
        article_centiment_like = driver.find_elements_by_css_selector(
            '#mArticle > div.foot_view > div.emotion_wrap > div.emotion_list > div > div > div > div.selectionbox.type-LIKE.unselected > span.count')[0].text
        time.sleep(3)
    except:
        article_centiment_like = ' '
        pass
    
    time.sleep(3)
        
    try:
        article_centiment_angry = driver.find_elements_by_css_selector(
            '#mArticle > div.foot_view > div.emotion_wrap > div.emotion_list > div > div > div > div.selectionbox.type-ANGRY.unselected > span.count')[0].text
        time.sleep(3)        
    except:
        article_centiment_angry = ' '
        pass
        
    return article_centiment_like, article_centiment_angry


In [17]:
urls_list = []
dates_list= []
heads_list = []
press_list = []
contents_list = []
centiments_like_list, centiments_angry_list = [], []

for k in tqdm(url):
    driver.get(k)
    
    urls_list = sum( [urls_list, get_article_url()], [])
    
for article_url in urls_list:
    
    driver.get(article_url)    
    contents_list.append(get_article_contents())
    time.sleep(1)
    heads_list.append(get_article_head())
    time.sleep(1)
    dates_list.append(get_article_date())
    time.sleep(1)
    press_list.append(get_press_name())
    time.sleep(1)
    tmp1, tmp2 = get_article_centiments()
    centiments_like_list.append(tmp1)
    centiments_angry_list.append(tmp2)
    time.sleep(3)
    
import pandas as pd

df = pd.DataFrame([dates_list, press_list, heads_list, contents_list, centiments_like_list, centiments_angry_list, urls_list]).T
df.columns = ['date', 'press', 'head', 'contents', 'like count', 'angry count', 'url']

df['like count'] = df['like count'].replace('-', '0')
df['angry count'] = df['angry count'].replace('-', '0')

comments_list = []
comments_num = []

for article_url in tqdm(urls_list):
    driver.get(article_url)
    print(driver.current_url)

    try:
        num_imsi = driver.find_element_by_xpath('//*[@id="alex-header"]/em').text
        comments_num.append(num_imsi) 
        time.sleep(3)
    except:
        comments_num.append('0')
    
    if num_imsi == '0':
        comments_list.append('no comments')
        time.sleep(1)
        continue
    
    cnt = 0
    
    try:
        if driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > button').text != '':
            while cnt < 2:
                driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > button').click()
                cnt += 1
            time.sleep(5)
    except:
        print('there is no More button for comments')
        pass
    
    try: 
        html = driver.page_source
        dom = bs(html, 'lxml')
        comments = dom.find_all("p", "desc_txt font_size_17", limit= 10) 
        comments_list.append(comments)
        time.sleep(3)
    except:
        print('comments text crawling error')
        comments_list.append(' ')
        pass
    
df['comments count'] = comments_num
df['comments'] = comments_list
df

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

https://news.v.daum.net/v/20200102075218653
there is no More button for comments


 11%|█████████▎                                                                          | 1/9 [00:15<02:00, 15.07s/it]

https://news.v.daum.net/v/20200102120036824


 22%|██████████████████▋                                                                 | 2/9 [00:18<01:21, 11.58s/it]

...

https://entertain.v.daum.net/v/20200105235236310


 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [01:17<00:10, 10.87s/it]

https://news.v.daum.net/v/20200105205037906
there is no More button for comments


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:32<00:00, 10.30s/it]


,date,press,head,contents,like count,angry count,url,comments count,comments
0,2020. 01. 02. 07:52,MBC,[스마트 리빙] 미세먼지 심한 날엔 독감 더 잘 걸려요,[뉴스투데이] 미세먼지가 심한 날에는 독감과 같은 호흡기 감염 질환에 걸리지 않도록...,0,0,http://v.media.daum.net/v/20200102075218653,1,"[<p class=""desc_txt font_size_17"" data-reactid..."
1,2020. 01. 02. 12:00,뉴시스,"소비자기본법 40주년..공정위, 아이디어 공모전·학술대회 개최",[세종=뉴시스] 강종민 기자 = 정부세종청사 공정거래위원회. ppkjm@newsis...,0,0,http://v.media.daum.net/v/20200102120036824,0,
2,2020. 01. 02. 17:47,뉴스1,中 '폐렴 미스터리' WHO도 조사 나서..숙주는 야생동물?,지난달 31일 원인불명의 바이러스성 폐렴이 발생해 휴업 명령이 내려진 중국 우한시 ...,0,0,http://v.media.daum.net/v/20200102174744517,11,"[<p class=""desc_txt font_size_17"" data-reactid..."
3,2020. 01. 03. 09:19,머니S,중국 폐렴 확산.. 제2의 사스 악몽 '공포',지난달 31일 중국의 한 해산물 시장에서 바이러스성 폐렴이 집단발병해 7명이 중태에...,0,0,http://v.media.daum.net/v/20200103091902676,0,
4,2020. 01. 04. 00:05,쿠키뉴스,中원인불명 폐렴에 홍콩 비상경보.. 우리는 中정보만 믿고 대응?,사진=CGTN 방송 화면 갈무리\n최근 중국 후베이 성 우한 시의 화난 수산시장에서...,0,0,http://v.media.daum.net/v/20200104000505138,0,
5,2020. 01. 05. 15:42,스포츠동아,"'미스트롯' 측 ""오늘(5일) 의정부 공연취소, 구조안전상 문제"" [공식입장]","[동아닷컴]\n‘미스트롯’ 측 “오늘(5일) 의정부 공연취소, 구조안전상 문제”\n...",0,0,http://v.media.daum.net/v/20200105154228658,0,
6,2020. 01. 05. 16:33,스포츠조선,"[공식] '미스트롯' 전국투어, 5일 의정부 공연 안전상 문제로 취소 ""깊은 사과""",[스포츠조선 백지은 기자] '미스트롯' 전국투어 콘서트 시즌2 '청춘'(이하 '미스...,0,0,http://v.media.daum.net/v/20200105163342087,0,
7,2020. 01. 05. 23:52,세계일보,"송가인, 미스트롯 의정부 공연 전격 취소에 ""미안하고 감사하다""",\n대세 가수 송가인(사진·본명 조은심)이 5일 오후로 예정됐던 ’내일은 미스트롯...,0,0,http://v.media.daum.net/v/20200105235236310,0,
8,2020. 01. 05. 20:50,동아사이언스,"중 후베이성 우한 폐렴 사태..WHO 주시, 전문가들 신종바이러스 우려",중국 후베이성 성도 우한. 크리에이티브커먼스 제공\n중국 후베이성 성도 우한에서 원...,0,0,http://v.media.daum.net/v/20200105205037906,6,"[<p class=""desc_txt font_size_17"" data-reactid..."


In [18]:
xlsx_file_name = '다음뉴스_코로나_2021년_7월_크롤링_{}.xlsx'.format(date)
df.to_excel(xlsx_file_name)

In [15]:
def get_url_list():
    for i in range(20210801, 20210832):
        etl_ymd = str(i)
        url.append('https://search.daum.net/search?nil_suggest=btn&w=news&DA=PGD&q=%EC%BD%94%EB%A1%9C%EB%82%98&p=1&period=u&sd='+etl_ymd+'000000&ed='+etl_ymd+'235959')
    return url

url = []
get_url_list()

def get_article_url():
    
    urls_list = []
    
    for i in tqdm(range(1, 11)):
        try:
            order_num = str(i)
            article_url = driver.find_elements_by_xpath('//*[@id="newsColl"]/div[1]/ul/li['+order_num+']/div[2]/span[1]/a')
            urls_list.append(article_url[0].get_attribute('href'))    
            time.sleep(3)

        except:
            continue
        
    return urls_list

heads_list = []

def get_article_head():
    
    try:
        article_head = driver.find_elements_by_css_selector('#cSub > div > h3')[0].text
        time.sleep(1)
    except:
        article_head = ' '
        pass
    
    return article_head

dates_list = []

def get_article_date():
    
    try:
        html = driver.page_source
        dom = bs(html, 'lxml')
        article_date = dom.find("span", "num_date").text 
        time.sleep(1)
    except:
        article_date = ' '
        pass
    
    return article_date

press_list = []

def get_press_name():

    try:
        press_name = driver.find_elements_by_css_selector('#cSub > div > em > a > img')[0].get_attribute("alt")
        time.sleep(1)
    except:
        press_name = ' '
        pass
    
    return press_name

contents_list = []

def get_article_contents():
    
    try:
        article_content = driver.find_elements_by_xpath('//*[@id="harmonyContainer"]')[0].text
        time.sleep(1)
    except:
        article_content = ' '
        pass
        
    return article_content

centiments_like_list, centiments_angry_list = [], []

def get_article_centiments():
        
    try:
        article_centiment_like = driver.find_elements_by_css_selector(
            '#mArticle > div.foot_view > div.emotion_wrap > div.emotion_list > div > div > div > div.selectionbox.type-LIKE.unselected > span.count')[0].text
        time.sleep(3)
    except:
        article_centiment_like = ' '
        pass
    
    time.sleep(3)
        
    try:
        article_centiment_angry = driver.find_elements_by_css_selector(
            '#mArticle > div.foot_view > div.emotion_wrap > div.emotion_list > div > div > div > div.selectionbox.type-ANGRY.unselected > span.count')[0].text
        time.sleep(3)        
    except:
        article_centiment_angry = ' '
        pass
        
    return article_centiment_like, article_centiment_angry


In [ ]:
urls_list = []
dates_list= []
heads_list = []
press_list = []
contents_list = []
centiments_like_list, centiments_angry_list = [], []

for k in tqdm(url):
    driver.get(k)
    
    urls_list = sum( [urls_list, get_article_url()], [])
    
for article_url in urls_list:
    
    driver.get(article_url)    
    contents_list.append(get_article_contents())
    time.sleep(1)
    heads_list.append(get_article_head())
    time.sleep(1)
    dates_list.append(get_article_date())
    time.sleep(1)
    press_list.append(get_press_name())
    time.sleep(1)
    tmp1, tmp2 = get_article_centiments()
    centiments_like_list.append(tmp1)
    centiments_angry_list.append(tmp2)
    time.sleep(3)
    
import pandas as pd

df = pd.DataFrame([dates_list, press_list, heads_list, contents_list, centiments_like_list, centiments_angry_list, urls_list]).T
df.columns = ['date', 'press', 'head', 'contents', 'like count', 'angry count', 'url']

df['like count'] = df['like count'].replace('-', '0')
df['angry count'] = df['angry count'].replace('-', '0')

comments_list = []
comments_num = []

for article_url in tqdm(urls_list):
    driver.get(article_url)
    print(driver.current_url)

    try:
        num_imsi = driver.find_element_by_xpath('//*[@id="alex-header"]/em').text
        comments_num.append(num_imsi) 
        time.sleep(3)
    except:
        comments_num.append('0')
    
    if num_imsi == '0':
        comments_list.append('no comments')
        time.sleep(1)
        continue
    
    cnt = 0
    
    try:
        if driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > button').text != '':
            while cnt < 2:
                driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > button').click()
                cnt += 1
            time.sleep(5)
    except:
        print('there is no More button for comments')
        pass
    
    try: 
        html = driver.page_source
        dom = bs(html, 'lxml')
        comments = dom.find_all("p", "desc_txt font_size_17", limit= 10) 
        comments_list.append(comments)
        time.sleep(3)
    except:
        print('comments text crawling error')
        comments_list.append(' ')
        pass
    
df['comments count'] = comments_num
df['comments'] = comments_list
df

In [ ]:
xlsx_file_name = '다음뉴스_코로나_2021년_8월_크롤링_{}.xlsx'.format(date)
df.to_excel(xlsx_file_name)

In [15]:
def get_url_list():
    for i in range(20210901, 20210931):
        etl_ymd = str(i)
        url.append('https://search.daum.net/search?nil_suggest=btn&w=news&DA=PGD&q=%EC%BD%94%EB%A1%9C%EB%82%98&p=1&period=u&sd='+etl_ymd+'000000&ed='+etl_ymd+'235959')
    return url

url = []
get_url_list()

def get_article_url():
    
    urls_list = []
    
    for i in tqdm(range(1, 11)):
        try:
            order_num = str(i)
            article_url = driver.find_elements_by_xpath('//*[@id="newsColl"]/div[1]/ul/li['+order_num+']/div[2]/span[1]/a')
            urls_list.append(article_url[0].get_attribute('href'))    
            time.sleep(3)

        except:
            continue
        
    return urls_list

heads_list = []

def get_article_head():
    
    try:
        article_head = driver.find_elements_by_css_selector('#cSub > div > h3')[0].text
        time.sleep(1)
    except:
        article_head = ' '
        pass
    
    return article_head

dates_list = []

def get_article_date():
    
    try:
        html = driver.page_source
        dom = bs(html, 'lxml')
        article_date = dom.find("span", "num_date").text 
        time.sleep(1)
    except:
        article_date = ' '
        pass
    
    return article_date

press_list = []

def get_press_name():

    try:
        press_name = driver.find_elements_by_css_selector('#cSub > div > em > a > img')[0].get_attribute("alt")
        time.sleep(1)
    except:
        press_name = ' '
        pass
    
    return press_name

contents_list = []

def get_article_contents():
    
    try:
        article_content = driver.find_elements_by_xpath('//*[@id="harmonyContainer"]')[0].text
        time.sleep(1)
    except:
        article_content = ' '
        pass
        
    return article_content

centiments_like_list, centiments_angry_list = [], []

def get_article_centiments():
        
    try:
        article_centiment_like = driver.find_elements_by_css_selector(
            '#mArticle > div.foot_view > div.emotion_wrap > div.emotion_list > div > div > div > div.selectionbox.type-LIKE.unselected > span.count')[0].text
        time.sleep(3)
    except:
        article_centiment_like = ' '
        pass
    
    time.sleep(3)
        
    try:
        article_centiment_angry = driver.find_elements_by_css_selector(
            '#mArticle > div.foot_view > div.emotion_wrap > div.emotion_list > div > div > div > div.selectionbox.type-ANGRY.unselected > span.count')[0].text
        time.sleep(3)        
    except:
        article_centiment_angry = ' '
        pass
        
    return article_centiment_like, article_centiment_angry


In [ ]:
urls_list = []
dates_list= []
heads_list = []
press_list = []
contents_list = []
centiments_like_list, centiments_angry_list = [], []

for k in tqdm(url):
    driver.get(k)
    
    urls_list = sum( [urls_list, get_article_url()], [])
    
for article_url in urls_list:
    
    driver.get(article_url)    
    contents_list.append(get_article_contents())
    time.sleep(1)
    heads_list.append(get_article_head())
    time.sleep(1)
    dates_list.append(get_article_date())
    time.sleep(1)
    press_list.append(get_press_name())
    time.sleep(1)
    tmp1, tmp2 = get_article_centiments()
    centiments_like_list.append(tmp1)
    centiments_angry_list.append(tmp2)
    time.sleep(3)
    
import pandas as pd

df = pd.DataFrame([dates_list, press_list, heads_list, contents_list, centiments_like_list, centiments_angry_list, urls_list]).T
df.columns = ['date', 'press', 'head', 'contents', 'like count', 'angry count', 'url']

df['like count'] = df['like count'].replace('-', '0')
df['angry count'] = df['angry count'].replace('-', '0')

comments_list = []
comments_num = []

for article_url in tqdm(urls_list):
    driver.get(article_url)
    print(driver.current_url)

    try:
        num_imsi = driver.find_element_by_xpath('//*[@id="alex-header"]/em').text
        comments_num.append(num_imsi) 
        time.sleep(3)
    except:
        comments_num.append('0')
    
    if num_imsi == '0':
        comments_list.append('no comments')
        time.sleep(1)
        continue
    
    cnt = 0
    
    try:
        if driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > button').text != '':
            while cnt < 2:
                driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > button').click()
                cnt += 1
            time.sleep(5)
    except:
        print('there is no More button for comments')
        pass
    
    try: 
        html = driver.page_source
        dom = bs(html, 'lxml')
        comments = dom.find_all("p", "desc_txt font_size_17", limit= 10) 
        comments_list.append(comments)
        time.sleep(3)
    except:
        print('comments text crawling error')
        comments_list.append(' ')
        pass
    
df['comments count'] = comments_num
df['comments'] = comments_list
df

In [ ]:
xlsx_file_name = '다음뉴스_코로나_2021년_9월_크롤링_{}.xlsx'.format(date)
df.to_excel(xlsx_file_name)